In [ ]:
import os
import numpy as np
from tqdm import tqdm
import tifffile as tif
from scipy.ndimage.interpolation import rotate
from tensorflow.keras.utils import to_categorical

In [ ]:
def import_tif(file):
    """Function to import numpy file and take the np.abs of the data
    
    Parameters
    ==========
    
    file - (str)
        the directory path on where the .tif file image is
        
    
    Returns
    =======
    
    the data held inside the tif file designated by the User
    
    """
    diff3D = tif.imread(file).astype(np.float32)
    return diff3D


def rot90(m, k=1, axis=2):
    """Rotate an array k*90 degrees in the counter-clockwise direction around the given axis
    
    
    """
    m = np.swapaxes(m, 2, axis)
    m = np.rot90(m, k)
    m = np.swapaxes(m, 2, axis)
    return m

def rotations24(polycube):
    """Obtain all the 24 different projections of a cube
    
    """
    # imagine shape is pointing in axis 0 (up)

    # 4 rotations about axis 0
    first = rotations4(polycube, 0)

    # rotate 180 about axis 1, now shape is pointing down in axis 0
    # 4 rotations about axis 0
    second = rotations4(rot90(polycube, 2, axis=1), 0)

    # rotate 90 or 270 about axis 1, now shape is pointing in axis 2
    # 8 rotations about axis 2
    third = rotations4(rot90(polycube, axis=1), 2)
    fourth = rotations4(rot90(polycube, -1, axis=1), 2)

    # rotate about axis 2, now shape is pointing in axis 1
    # 8 rotations about axis 1
    fifth =  rotations4(rot90(polycube, axis=2), 1)
    sixth =  rotations4(rot90(polycube, -1, axis=2), 1)
    
    return np.concatenate((first, second, third, fourth, fifth, sixth))

def rotations4(polycube, axis):
    """List the four rotations of the given cube about the given axis."""
    master = []
    
    for i in range(4):
        master.append(rot90(polycube, i, axis))
        
    return master


def slice3D_easy(dataset, labels, diff3D, n_projections, label, Nc=64):
    """Cut slices out of the center of the 3D array
    
    Parameters
    ==========
    
    dataset - (np.array)
        the dataset array you would like to append the data to
    
    labels - (np.array)
        the labels array you would like to append the label to 
    
    diff3D - (np.array)
        the current 3D diffraction image to take slices from
    
    label - (int)
        the label of the dataset you are inputing   
    

    Returns
    =======
    Nothing
    """
    
    #Slice 3D array into specified 2D slices - lower theta rotation range from higher theta rotation range
    thetas = np.linspace(-45, 45, n_projections, endpoint=False)

    for theta in thetas:
        slice2D = rotate(diff3D, theta, axes=(0%3, (0+1)%3),
                         reshape=False, order=1, mode='constant')[:, int(Nc/2)]

        dataset.append(slice2D)
        labels.append(label)

In [ ]:
def loading_and_slice(all_files, save_dataset, save_labels, label,
                      number_of_slice_per_90_degree=5, Nc=128):
    
    """Load all data and slices into numpy array
    
    Parameters
    ----------
    all_files : list
        list of file names to import
        
    save_dataset : array
        Array to auto append data to - can be empty
    
    save_labels : array
        array to atuto append labels to - can be empty
        
    label : int
        label to give the files
        
    number_of_slice_per_90_degree : 5
        
    Nc : int
        image dimension
    """
    
    for file in tqdm(all_files, total=len(all_files)):

        # importing the raw data files
        data = import_tif(file)

        # Raw data is a NxNxN cube
        # Obtain all 24 cube rotations for the data
        crystal_rotations = rotations24(data)
        for crystal in crystal_rotations:    
            slice3D_easy(save_dataset, save_labels, crystal,
                         number_of_slice_per_90_degree , label, Nc=image_size)
            



In [ ]:
# Iteratable list of flists
all_files_list = [flist1, flist2, flist3]

# Iterable list of lables
labels_list = [0, 1, 2]

# Blank lists to save to
save_dataset = []
save_labels = []

for label, all_files in zip(labels_list, all_files_list):
    loading_and_slice(all_files, save_dataset, save_labels, label,
                      number_of_slice_per_90_degree=5, Nc=128):

In [ ]:
# Only use the central 64 x 64 pixles of the images
save_dataset = save_dataset[:, int(Nc/2)-32:int(Nc/2)+32, int(Nc/2)-32:int(Nc/2)+32]

In [ ]:
# Normalize Data Later
np.save('/testiing_data/test_c2_128x128_dataset_01.npy', save_dataset)
np.save('/testing_data/test_c2_128x128_labels_01.npy', save_labels)

In [ ]:
def add_train_noise(img, VARIABILITY):
    """Add random poisson noise to an image
    
    Parameters
    ==========
    img (np) - 2D numpy image
    
    VARIABILITY - int - the value to be passed into the np.random.poisson function
    
    """
    mins = np.min(img)
    maxs = np.max(img)

    PEAK = VARIABILITY

    img = np.random.poisson(img / maxs * PEAK) / PEAK * maxs

    # Clip data so there are no negative numbers
    img = np.clip(img, mins, maxs)
        
    return img

In [ ]:
X_data = np.load('/testing_data/test_c2_128x128_dataset_01.npy')
y_data = np.load('/testing_data/test_c2_128x128_labels_01.npy')

# Add noise to the data
new_X_data = []
new_y_data = []

for ids, slices in tqdm(enumerate(X_data), total=len(X_data)):
    for value in [4000]:

        append_imgs = add_train_noise(slices.copy(), value)

        new_X_data.append(append_imgs)
        new_y_data.append(y_data[ids])
        
new_X_data = np.asarray(new_X_data)
new_y_data = np.asarray(new_y_data)

# The number of labels to categorize
num_of_labels = 3
new_y_data = to_categorical(new_y_data, num_of_labels)
        
print(new_X_data.shape, new_y_data.shape)

In [ ]:
np.save('/testing_data/noise_test_c14_128x128_dataset_01.npy', new_X_data)
np.save('/testing_data/noise_test_c14_128x128_labels_01.npy', new_y_data)